# POCA retrieval tutorial

cryoswath can, also, retrieve point of closest approach (POCA) elevation
estimates. Those are especially useful when studying ice caps, where the
highest points are ice covered. There are more potential use-cases for
POCA data; however, consider whether the implemented algorithm suits
your needs (if not, I encourage you to write a little module yourself;
I'll be happy to adopt it). The implementation, in a nutshell, finds the
maximum coherence on the leading edge (i.e., the first rise in signal
power) and interprets the location of the corresponding waveform sample
as POCA.

I acknowledge, **POCA's are somewhat hidden**. However, if you batch
process the tracks for any region, you'll be served the POCA's
automatically. It may be easiest to get them from a file on disk -- have
a look in `data/L2_poca`. Currently, the alternative is to ask them from
the `l1b` module which requires you to loop over all tracks yourself.

In [ ]:
import sys
sys.path.append((".."))

In [ ]:
from cryoswath.misc import *
from cryoswath import gis, l1b, l2, l3
from cryoswath.test_plots import waveform
import os
import geopandas as gpd

## Viewing point data

To show an example, first, some tracks are viewed to select a track ID/time stamp.

In [ ]:
load_cs_ground_tracks("06-01", "2024-01", "2024-02", buffer_region_by=10_000)

Next, the L1b data of the first track is loaded.

In [ ]:
l1b_data = l1b.L1bData.from_id("2024-01-02 10:53:42")
l1b_data

Then, the data is processed - meaning geolocating the echo origins.

In [ ]:
l1b_data = l1b_data.append_best_fit_phase_index()

You can obtain a list of all waveforms for which a POCA was found as follows:

In [ ]:
poca_wf_time_idx = l1b_data.time_20_ku[~l1b_data.poca_idx.isnull()]
poca_wf_time_idx

To view the first waveform that contains the POCA, use the crosssection test plot. 

In [ ]:
# fyi: double square brackets to provide list as required by function
waveform.dem_transect(l1b_data.sel(time_20_ku=poca_wf_time_idx[[0]]))

Let's briefly view the elevation differences to the reference DEM as a histogramm.

In [ ]:
# the many `.sel` successively cut down the data to the requested
# fraction. 1. select phase wrap factor, 2. select waveforms containing
# POCAs (time_20_ku=...), 3. select POCA sample
l1b_data.sel(phase_wrap_factor=l1b_data.ph_idx).sel(time_20_ku=poca_wf_time_idx).sel(ns_20_ku=l1b_data.sel(time_20_ku=poca_wf_time_idx).poca_idx).xph_elev_diffs.plot.hist(bins=range(-50, 51));

## POCA product on a larger scale

Usually, you will care less about individual waveforms. Here, it is
shown how to do the processing on a more general level. Start with
processing all the tracks of January 2024 for Iceland (or make your own
choice). This step produces `.feather` files in `./data/L2_poca`, using
the default setup. Those will contain the POCA data and can be read,
e.g., by `geopandas.read_feather()`. You can actually skip this step,
but in many use-cases, e.g., processing data for longer periods, it is
convenient to separate downloading and processessing the data from
further analysis. This is because one may want to use multiple processes
for the download (I recommend 8, respecting ESA's restrictions) and for
processing (can be specified should 8 be too many for your local
workstation). Much of the analysis profits from sharing working memory,
i.e., using a single process (except for advanced tools).

In [ ]:
l2.process_and_save("06-01", "2024-01", "2024-02", buffer_region_by=0)

For this example, the cell below collects all the POCA data. If you
chose to skip the step above, it will, first, download and process the
data, of course.

In [ ]:
poca_data = l2.from_id(load_cs_ground_tracks("06-01", "2024-01", "2024-02", buffer_region_by=0).index, reprocess=False)[1]
poca_data
# all_l2_data = l2.from_id(load_cs_ground_tracks("06-01", "2024-01", "2024-02", buffer_region_by=0).index, reprocess=False)

And, finally, below a histogram is shown.

In [ ]:
poca_data.h_diff.plot.hist(bins=range(-50, 51))